# Test the predictions of the model on a sample melody

In [1]:
import sys; sys.path.insert(0, "..")
from AugmentedNet.score_parser import parseScore
from AugmentedNet.input_representations import Bass19
from AugmentedNet.output_representations import available_representations as availableOutputs
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

Loading the melody

In [3]:
# https://gist.githubusercontent.com/napulen/60cb786a83efc1753baccc9b42a10072/raw/bf3d0c62993c63aec0a498a83a9c924b02523b4c/rachmaninoff3.musicxml
melodyExample = "./rachmaninoff3.mxl"

In [4]:
df = parseScore(melodyExample)

ConverterException: File not found or no such format found for: ./rachmaninoff3.mxl

In [ ]:
melodyInput = Bass19(df)

In [ ]:
plt.figure(figsize=(25,10))
plt.pcolormesh(melodyInput.array[:100].T, edgecolors="k")

Loading the pre-trained model

In [ ]:
modelPath = ".model_checkpoint/validationset/bps-lr01-0001-210530T233203/55-4.109-0.7956.hdf5"
model = keras.models.load_model(modelPath)

In [ ]:
outputLayers = [l.name.split("/")[0] for l in model.outputs]

In [ ]:
inputProcessed = melodyInput.array[:640].reshape(1, 640, 19)
modelInput = [inputProcessed, inputProcessed]
inputProcessed.shape

In [ ]:
predictions = model.predict(modelInput)

In [ ]:
dfdict = {}
for outputRepr, pred in zip(outputLayers, predictions):
    print(outputRepr, pred.shape)
    predOnehot = np.argmax(pred[0], axis=1).reshape(-1, 1)
    decoded = availableOutputs[outputRepr].decode(predOnehot) 
    dfdict[outputRepr] = decoded
df = pd.DataFrame(dfdict)

In [ ]:
pd.set_option('display.max_rows', 640)
df

In [ ]:
df.to_csv("rachmaninoff_analysis.csv")